<a href="https://colab.research.google.com/github/rulas99/thermal_anomaly_detection/blob/main/thermal_anomalies_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Semantic Segmentation [source](https://towardsdatascience.com/train-neural-net-for-semantic-segmentation-with-pytorch-in-50-lines-of-code-830c71a6544f)

In [1]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

from pandas import read_csv

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
Learning_Rate = 1e-5
width = 500
height = 500 # image width and height
batchSize = 7

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True) # Load net

Net.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 3 classes

Net = Net.to(device) # Set net to GPU or CPU

optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

weights = torch.FloatTensor([0.1,1]).cuda()
criterion = torch.nn.CrossEntropyLoss(weight = weights) # Set loss function

In [7]:
train=read_csv("/content/drive/MyDrive/segment_sentinel2_hotspot/train_2.csv")

In [8]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),])
                         #tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
                         
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width),tf.InterpolationMode.NEAREST),tf.ToTensor()])

In [14]:
def ReadRandomImage(): # First lets load random image and  the corresponding annotation
    idx=np.random.randint(0,len(train)) # Select random image
    thermal = cv2.imread(train.ori_path.iloc[idx])
    masked =  cv2.imread(train.label_path.iloc[idx],0)
    AnnMap = np.zeros(thermal.shape[0:2],np.float32)
    if masked is not None:  AnnMap[ masked == 255 ] = 1
    Img=transformImg(thermal)
    AnnMap=transformAnn(AnnMap)
    return Img,AnnMap


def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])
    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()
    return images, ann

In [ ]:
for itr in range(300): # Training loop
   images,ann=LoadBatch() # Load taining batch
   images=torch.autograd.Variable(images,requires_grad=False).to(device) # Load image
   ann = torch.autograd.Variable(ann, requires_grad=False).to(device) # Load annotation
   Pred=Net(images)['out'] # make prediction
   optimizer.zero_grad()
   Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
   Loss.backward() # Backpropagate loss
   optimizer.step() # Apply gradient descent change to weight

   if itr % 20 == 0: #Save model weight once every 100 steps permenant file
        accG = [(torch.argmax(Pred[i], 0).cpu().detach().numpy() == ann[i].cpu().detach().numpy()).sum()/(width*height) for i in range(batchSize)]
        accM = round(sum(accG)/batchSize,5)
        loss = round(float(Loss.data.cpu()),5)
        print(itr,f") Loss = {loss} -- Accuracy = {accM}")
        print(f"Saving model_{itr}.torch")
        torch.save(Net.state_dict(),   f'/content/drive/MyDrive/segment_sentinel2_hotspot/Models_2/model_{itr}.torch')

0 ) Loss = 0.74058 -- Accuracy = 0.26743
Saving model_0.torch
20 ) Loss = 0.72773 -- Accuracy = 0.4052
Saving model_20.torch
40 ) Loss = 0.71368 -- Accuracy = 0.40753
Saving model_40.torch
60 ) Loss = 0.68198 -- Accuracy = 0.54018
Saving model_60.torch
80 ) Loss = 0.65246 -- Accuracy = 0.7413
Saving model_80.torch
100 ) Loss = 0.61761 -- Accuracy = 0.83764
Saving model_100.torch
120 ) Loss = 0.56387 -- Accuracy = 0.96291
Saving model_120.torch
140 ) Loss = 0.48335 -- Accuracy = 0.98987
Saving model_140.torch
160 ) Loss = 0.44075 -- Accuracy = 0.98601
Saving model_160.torch
180 ) Loss = 0.41747 -- Accuracy = 0.99519
Saving model_180.torch
200 ) Loss = 0.38832 -- Accuracy = 0.99498
Saving model_200.torch
220 ) Loss = 0.3557 -- Accuracy = 0.99751
Saving model_220.torch
240 ) Loss = 0.34138 -- Accuracy = 0.995
Saving model_240.torch
260 ) Loss = 0.33476 -- Accuracy = 0.99433
Saving model_260.torch
